### functon 1 讀取檔案
input ： path, is_display(bool)


output： 物件

In [52]:
import os
from OCC.Display.SimpleGui import init_display
from OCC.Extend.DataExchange import read_iges_file

def read_file(path, is_display=False):
    
    if os.path.exists(path):           # 防止路徑不存在
        shapes = read_iges_file(path)  # 一定要用這個方法讀取
    else:
        print("檔案路徑錯誤！")
    
    if is_display:
        # 初始化 3D 顯示環境
        display, start_display, add_menu, add_function_to_menu = init_display()
        display.DisplayShape(shapes, update=True)
        start_display()
        
    return shapes

In [74]:
path = r"C:\alan\ITRI\ITRI-test\model\simple_3.IGS"#物件路徑
object_1 = read_iges_file(path)
print("type:",type(object_1))  # 应输出 <class 'OCC.Core.TopoDS.TopoDS_Shape'>
object_1=object_1[0]#如果type(model)=<class 'list'>就要加這行，正常應該是type: <class 'OCC.Core.TopoDS.TopoDS_Compound'>
print("type:",type(object_1))  # 应输出 <class 'OCC.Core.TopoDS.TopoDS_Shape'>

type: <class 'list'>
type: <class 'OCC.Core.TopoDS.TopoDS_Compound'>


### functon 2 分割物件平面
input ： object, save_path, is_save(bool)

output： 面 1 ~ 面 n

In [75]:
import os
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopoDS import topods
from OCC.Core.TopAbs import TopAbs_FACE
from OCC.Display.OCCViewer import OffscreenRenderer

# 平面影像儲存路徑
image_directory = r"C:\alan\ITRI\IRTI_path_generation\Images"

def plane_segmentation(object, save_path, is_save=True):
    
    if not os.path.exists(save_path):        # 如果 save_path 不存在，則創建一個
        os.makedirs(save_path)
        
    
    explorer = TopExp_Explorer(object, TopAbs_FACE) # TopAbs_FACE 只對面感興趣
    faces = []
    index = 1
    # 宣告一個渲彩器(不會顯示出來)
    renderer = OffscreenRenderer()
    renderer.Create()
    
    while explorer.More():
        
        face = topods.Face(explorer.Current())
        faces.append(face)                     # 將當前的面儲存到 list 中
        
        renderer.EraseAll()
        renderer.DisplayShape(face, update=True, color='BLUE', transparency=0.5)  # 可以調整顏色和透明度
        renderer.FitAll()
        
        if is_save:
            image_path = os.path.join(image_directory, f'object_face_{index}.png')            
            renderer.View.Dump(image_path)
            print(f'Saved image of face {index} to {image_path}')

        index += 1
        explorer.Next()                      # 移動到下一個面
    
    return faces

In [76]:
object_1_faces = plane_segmentation(object_1, image_directory)
print("object_1 總共有", len(object_1_faces), "個物件")

Many colors for color name BLUE, using first.
OffscreenRenderer content dumped to c:\alan\ITRI\IRTI_path_generation\程式碼\capture-1-1718613562.jpeg
Saved image of face 1 to C:\alan\ITRI\IRTI_path_generation\Images\object_face_1.png
Many colors for color name BLUE, using first.
OffscreenRenderer content dumped to c:\alan\ITRI\IRTI_path_generation\程式碼\capture-2-1718613562.jpeg
Saved image of face 2 to C:\alan\ITRI\IRTI_path_generation\Images\object_face_2.png
Many colors for color name BLUE, using first.
OffscreenRenderer content dumped to c:\alan\ITRI\IRTI_path_generation\程式碼\capture-3-1718613562.jpeg
Saved image of face 3 to C:\alan\ITRI\IRTI_path_generation\Images\object_face_3.png
Many colors for color name BLUE, using first.
OffscreenRenderer content dumped to c:\alan\ITRI\IRTI_path_generation\程式碼\capture-4-1718613562.jpeg
Saved image of face 4 to C:\alan\ITRI\IRTI_path_generation\Images\object_face_4.png
Many colors for color name BLUE, using first.
OffscreenRenderer content dumped t

### functon 3 平面+線段 生成 drive plane
input ： 平面 選擇的邊線 

output： 面 1 ~ 面 n

In [77]:
## 計算線段曲率
from OCC.Core.BRepLProp import BRepLProp_CLProps
from OCC.Core.BRepAdaptor import BRepAdaptor_Curve
from OCC.Core.GeomAPI import GeomAPI_ProjectPointOnSurf
from OCC.Core.GCPnts import GCPnts_UniformAbscissa
from OCC.Core.GCPnts import GCPnts_AbscissaPoint

from OCC.Core.GeomLProp import GeomLProp_SLProps
from OCC.Core.gp import gp_Pnt, gp_Vec

def calculate_average_curvature(edge, curve_length, props, point_spacing, num_points=10):
    # 將交線轉換成曲線
    curve_adaptor = BRepAdaptor_Curve(edge)
    first_param = curve_adaptor.FirstParameter() 
    last_param = curve_adaptor.LastParameter()   
    num_points = int(curve_length / point_spacing) + 1
    curvature_sum = 0
    
    for i in range(1, num_points - 1):  # 第一個跟最後一個點不算，怕有問題
        param = first_param + i * (last_param - first_param) * point_spacing / curve_length 
        point = curve_adaptor.Value(param)                                                 # 依序得到座標(從UV轉為XYZ)
        """
        它從你設計的這個面（比如一個彎曲的盾牌表面）中，取出它的數學模型，讓我們可以計算和操作它，例如測量它的彎曲程度或找到它的中心點等。
        """                                                 
        surface_handle = BRep_Tool.Surface(face)                                           
        projector = GeomAPI_ProjectPointOnSurf(point, surface_handle) ##在尋找如果從這個點垂直投影到曲面上，它會落在哪里
        
        if projector.NbPoints() > 0:  ##確保至少有一個投影點
            """
            在曲面上，每個點都可以通過兩個參數（通常表示為 u 和 v）來定位。這些參數有點像地圖上的經緯度，它們告訴你這個點在曲面的「網格」上的位置。
            """
            u, v = projector.LowerDistanceParameters()
        props.SetParameters(u, v)     ##告訴分析器在曲面上的哪個點進行計算
        if props.IsCurvatureDefined():
            curvature = props.MaxCurvature()
            
            
        curvature_sum += curvature    
        
    # if props.IsCurvatureDefined():
        #     max_dir, min_dir = gp_Dir(), gp_Dir()
        #     props.CurvatureDirections(max_dir, min_dir)
        #     # 最大最小曲率的方向?
        #     print("Direction of Maximum Curvature:", max_dir.X(), max_dir.Y(), max_dir.Z())
        #     print("Direction of Minimum Curvature:", min_dir.X(), min_dir.Y(), min_dir.Z())

    average_curvature = curvature_sum / (num_points - 2)  ## 計算平均曲率
    # print(face_index + "\t" + path_index + "\t" + average_curvature)  
    return average_curvature
    

In [78]:
from OCC.Core.GeomLProp import GeomLProp_SLProps
from OCC.Core.gp import gp_Pnt, gp_Vec
import math
def calculate_suitable_plane_spacing(edge, curve_length, props, point_spacing, num_points=10, x=0.5):
    # 將交線轉換成曲線
    curve_adaptor = BRepAdaptor_Curve(edge)
    first_param = curve_adaptor.FirstParameter() 
    last_param = curve_adaptor.LastParameter()   
    num_points = int(curve_length / point_spacing) + 1
    curvature_list = []
    
    for i in range(1, num_points - 1):  # 第一個跟最後一個點不算，怕有問題
        param = first_param + i * (last_param - first_param) * point_spacing / curve_length 
        point = curve_adaptor.Value(param)                                                 # 依序得到座標(從UV轉為XYZ)
        """
        它從你設計的這個面（比如一個彎曲的盾牌表面）中，取出它的數學模型，讓我們可以計算和操作它，例如測量它的彎曲程度或找到它的中心點等。
        """                                                 
        surface_handle = BRep_Tool.Surface(face)                                           
        projector = GeomAPI_ProjectPointOnSurf(point, surface_handle) ##在尋找如果從這個點垂直投影到曲面上，它會落在哪里
        
        if projector.NbPoints() > 0:  ##確保至少有一個投影點
            """
            在曲面上，每個點都可以通過兩個參數（通常表示為 u 和 v）來定位。這些參數有點像地圖上的經緯度，它們告訴你這個點在曲面的「網格」上的位置。
            """
            u, v = projector.LowerDistanceParameters()
        props.SetParameters(u, v)     ##告訴分析器在曲面上的哪個點進行計算
        if props.IsCurvatureDefined():
            curvature = props.MaxCurvature()
            
            
        curvature_list.append(curvature)    
        
    max_curvature = max(curvature_list) ## 得到曲線上的最大曲率
    if max_curvature == 0:              ## 反推得到曲率半徑
        min_radius = 1000           
    else:
        min_radius = 1. / max_curvature

    min_spacing = math.sin(math.acos(1-(x/min_radius))) * 2 * min_radius  ## 得到最大容許長度
    return min_spacing

In [58]:
## 讀取所有邊驗的功能
# from OCC.Core.TopoDS import topods_Edge
# from OCC.Core.TopAbs import TopAbs_EDGE
# from OCC.Core.BRepAdaptor import BRepAdaptor_Curve
# from OCC.Core.BRep import BRep_Tool

# def edges(face):
#     edge_explorer = TopExp_Explorer(face, TopAbs_EDGE)
#     edges = []
#     while edge_explorer.More():
        
#         edge = topods_Edge(edge_explorer.Current())
#         curve_adaptor = BRepAdaptor_Curve(edge)
#         if curve_adaptor.GetType() == 0:
#             selected_edge = edge
#             return selected_edge
#         edges.append(edge)
#         edge_explorer.Next()
    
#     return edges[0]

In [79]:
from OCC.Core.TopoDS import topods_Edge
from OCC.Core.TopAbs import TopAbs_EDGE
from OCC.Core.BRepAdaptor import BRepAdaptor_Curve
from OCC.Core.BRep import BRep_Tool
def edges(face):
    edge_explorer = TopExp_Explorer(face, TopAbs_EDGE)
    edges = []
    selected_edges = []  # 用于存储符合条件的线段
    
    while edge_explorer.More():
        edge = topods_Edge(edge_explorer.Current())
        curve_adaptor = BRepAdaptor_Curve(edge)
        if curve_adaptor.GetType() == 0:
            selected_edges.append(edge)  # 存储符合条件的线段
            if len(selected_edges) == 2:  # 如果找到了两条符合条件的线段
                return selected_edges[1]  # 返回第二条
        edges.append(edge)
        edge_explorer.Next()
    
    # 如果找不到两条符合条件的线段，则返回 None 或根据需要处理
    return None if len(selected_edges) < 2 else selected_edges[1]

In [80]:
from OCC.Core.gp import gp_Pnt, gp_Dir, gp_Pln, gp_Vec
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeFace
from OCC.Core.BRepAdaptor import BRepAdaptor_Curve
from OCC.Core.GCPnts import GCPnts_UniformAbscissa
from OCC.Display.SimpleGui import init_display
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Section
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopoDS import topods_Face
from OCC.Core.GCPnts import GCPnts_AbscissaPoint
from OCC.Core.GCPnts import GCPnts_UniformAbscissa  
from OCC.Core.BRepLProp import BRepLProp_SLProps
from OCC.Core.BRepAdaptor import BRepAdaptor_Surface
from OCC.Core.GeomAPI import GeomAPI_ProjectPointOnSurf
from OCC.Core.BRep import BRep_Tool
from OCC.Core.TopAbs import TopAbs_FACE, TopAbs_REVERSED

In [81]:
def calculate_section(face, length, curve_adaptor, name = '', plane_spacing = 10.0, point_spacing = 5.0, is_save=True, is_display=False):
    """
    可以更容易地獲取這個面的各種幾何信息，比如曲面的尺寸、形狀、位置等，並且可以更方便地執行一些複雜的操作，如計算曲率等。
    """
    adaptor_surface = BRepAdaptor_Surface(face, True) 
    props = BRepLProp_SLProps(adaptor_surface, 2, 0.01)  # 這個分析器被配置為能夠計算高達二階導數的幾何性質（這由第二個參數指定，此處為 2），這意味著它能夠計算曲率等性質
    num_planes = int(length / plane_spacing) + 1
    path_index = 0
    
    if length % plane_spacing != 0:
        iter_plane = num_planes + 1
    else:
        iter_plane = num_planes
    
    for i in range(0, iter_plane):
        path_index += 1
        u = curve_adaptor.FirstParameter() + i * plane_spacing
        if u >= curve_adaptor.LastParameter():
            u = curve_adaptor.LastParameter()
        pnt = curve_adaptor.Value(u)           ## 得到XYZ座標
        tangent_vector = curve_adaptor.DN(u, 1).Normalized() ##  tangent_vector。DN(u, 1) 表示取得曲線在 u 點的第一階導數（即切向量），並且通過 Normalized() 函數將此向量標準化
        tangent_dir = gp_Dir(tangent_vector)  #表示為一個方向，不具備長度
        
        perpendicular_plane = gp_Pln(pnt, tangent_dir)   ##定義一個幾何平面，用點跟法向量，無限大的平面
        
        # if is_display:
        #     perpendicular_face = BRepBuilderAPI_MakeFace(perpendicular_plane, -110, 110, -110, 110).Face()
        #     display.DisplayShape(perpendicular_face, update=True, color='RED', transparency=0.9)
            
        section = BRepAlgoAPI_Section(face, perpendicular_plane, False) #用於計算兩個B-Rep幾何實體之間交集的物件
        """
        具體來說，當你對兩個幾何形體（例如，兩個面）進行交集操作來查找它們的交界線時，ComputePCurveOn1(True) 指示算法在第一個面上計算這條交界線的投影。
        這意味著它將生成一個參數化曲線，描述交界線如何在第一個面的局部坐標系中行進
        """
        section.ComputePCurveOn1(True)
        section.Build()
        
        if section.IsDone() :
            intersection_shape = section.Shape()
            if is_display:
                display.DisplayShape(intersection_shape, update=True, color='GREEN')
                
            explorer = TopExp_Explorer(intersection_shape, TopAbs_EDGE)
            if not explorer.More():
                print("No edges found in the intersection shape.",face_index)
                continue
            edge = topods_Edge(explorer.Current())
            curve = BRepAdaptor_Curve(edge)    ## BRepAdaptor_Curve 是一個適配器類，用於從拓撲邊提取底層的曲線幾何資訊
                
            curve_length = GCPnts_AbscissaPoint.Length(curve, curve.FirstParameter(), curve.LastParameter())
            


            suitable_spacing = calculate_suitable_plane_spacing(edge, curve_length, props =props, point_spacing=1, num_points=1000) 
            print(str(face_index) + '\t' + str(path_index) + '\t' + "spacing:", suitable_spacing)
            if suitable_spacing < point_spacing:   ## 如果設定長度小於最大容許長度，則更改設定長度
                adjust_point_spacing = round(suitable_spacing-1)
            else:
                adjust_point_spacing = point_spacing
            # average_curvature = calculate_average_curvature(edge, curve_length, props =props, point_spacing=1.0, num_points=1000)
            # if average_curvature is not None:
            #     # print(str(face_index) + '\t' + str(path_index) + '\t' + "Average curvature:", average_curvature)
            # else:
            #     print("No valid points found to calculate curvature.")
            num_points = int(curve_length / adjust_point_spacing) + 1
            
            if curve_length % adjust_point_spacing != 0:
                iter_point = num_points + 1
            else:
                iter_point = num_points
    
            for j in range(0, iter_point):
                param = curve.FirstParameter() + j * (curve.LastParameter() - curve.FirstParameter()) * adjust_point_spacing / curve_length #因為有些UV被正規畫到0~1之間了
                if param >= curve.LastParameter():
                    param = curve.LastParameter()
                point = curve.Value(param)
                if is_display:
                    display.DisplayShape(point, update=True, color='YELLOW')
                
                surface_handle = BRep_Tool.Surface(face)
                projector = GeomAPI_ProjectPointOnSurf(point, surface_handle)
                
                if projector.NbPoints() > 0:
                    u, v = projector.LowerDistanceParameters()
                else:
                    print("No projection found for this point")
                    
                props.SetParameters(u, v)
                if props.IsCurvatureDefined():
                    normal = props.Normal()
                
                if face.Orientation() == TopAbs_REVERSED:
                    normal.Reverse()
                    
                if is_save:
                    with open(name  + '.txt', 'a') as outfile:
                        outfile.write(str(face_index) + '\t' + str(path_index) +
                                    '\t' + format(point.X(), ".2f") + '\t' + format(point.Y(), ".2f") + '\t' + format(point.Z(), ".2f") +
                                    '\t' + format(normal.X(), ".2f") + '\t' + format(normal.Y(), ".2f") + '\t' + format(normal.Z(), ".2f") + '\n')
        
    
                    with open(name + '.csv', 'a', newline='') as outfile:
                        # 寫入數據，使用逗號分隔
                        outfile.write(f"{face_index},{path_index}," +
                                    f"{format(point.X(), '.2f')},{format(point.Y(), '.2f')},{format(point.Z(), '.2f')}," +
                                    f"{format(normal.X(), '.2f')},{format(normal.Y(), '.2f')},{format(normal.Z(), '.2f')}\n")
        
    

In [90]:
face_index = 0
display, start_display, add_menu, add_function_to_menu = init_display()
display.DisplayShape(object_1, update=True)
for face in object_1_faces:
    face = topods_Face(face)
    print("Face orientation:", face.Orientation())
    face_index += 1
    
    selected_edge = edges(face) ################
    
    curve_adaptor = BRepAdaptor_Curve(selected_edge)
    length = curve_adaptor.LastParameter() - curve_adaptor.FirstParameter()
    # 指定的資料夾
    directory = r"C:\alan\ITRI\IRTI_path_generation\路徑結果"  # 更改为你的目录路径
    Name = "simple3_drive10"  # 更改为你的文件名

    
    calculate_section(face=face, length=length, curve_adaptor=curve_adaptor, name=os.path.join(directory, Name ), plane_spacing = 10.0, point_spacing = 15.0, is_save=True, is_display=True)
    
start_display()

C:\Users\sw_ya\AppData\Local\Temp\ipykernel_6840\1688185976.py:5: DeprecationWarning: Call to deprecated function topods_Face since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method topods.Face
  face = topods_Face(face)
C:\Users\sw_ya\AppData\Local\Temp\ipykernel_6840\144608263.py:11: DeprecationWarning: Call to deprecated function topods_Edge since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method topods.Edge
  edge = topods_Edge(edge_explorer.Current())
C:\Users\sw_ya\AppData\Local\Temp\ipykernel_6840\144608263.py:11: DeprecationWarning: Call to deprecated function topods_Edge since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method topods.Edge
  edge = topods_Edge(edge_explorer.Current())
C:\Users\sw_ya\AppData\Local\Temp\ipykernel_6840\1282987135.py:47: DeprecationWarning: Call to deprecated function topods_

Face orientation: 0
1	1	spacing: 63.23764701504603


C:\Users\sw_ya\AppData\Local\Temp\ipykernel_6840\1282987135.py:47: DeprecationWarning: Call to deprecated function topods_Edge since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method topods.Edge
  edge = topods_Edge(explorer.Current())


1	2	spacing: 63.23764701504603


C:\Users\sw_ya\AppData\Local\Temp\ipykernel_6840\1282987135.py:47: DeprecationWarning: Call to deprecated function topods_Edge since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method topods.Edge
  edge = topods_Edge(explorer.Current())


...

8	7	spacing: 8.888194417315589
8	8	spacing: 8.888194417315589


C:\Users\sw_ya\AppData\Local\Temp\ipykernel_6840\1282987135.py:47: DeprecationWarning: Call to deprecated function topods_Edge since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method topods.Edge
  edge = topods_Edge(explorer.Current())
C:\Users\sw_ya\AppData\Local\Temp\ipykernel_6840\1282987135.py:47: DeprecationWarning: Call to deprecated function topods_Edge since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method topods.Edge
  edge = topods_Edge(explorer.Current())


8	9	spacing: 8.888194417315589
8	10	spacing: 8.888194417315589


C:\Users\sw_ya\AppData\Local\Temp\ipykernel_6840\1282987135.py:47: DeprecationWarning: Call to deprecated function topods_Edge since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method topods.Edge
  edge = topods_Edge(explorer.Current())


8	11	spacing: 8.888194417315589


In [63]:

# face_index = 0
# display, start_display, add_menu, add_function_to_menu = init_display()
# display.DisplayShape(object_1, update=True)
# for face in object_1_faces:
#     face = topods_Face(face)
#     face_index += 1
#     adaptor_surface = BRepAdaptor_Surface(face, True)
#     props = BRepLProp_SLProps(adaptor_surface, 2, 0.01)
    
#     selected_edge = edges(face) ################
    
#     curve_adaptor = BRepAdaptor_Curve(selected_edge)
#     length = curve_adaptor.LastParameter() - curve_adaptor.FirstParameter()
    
#     num_planes = 5
#     spacing = length / (num_planes + 1)    
    
#     path_index = 0
    
#     for i in range(1, num_planes + 1):
#         path_index += 1
#         u = curve_adaptor.FirstParameter() + i * spacing
#         pnt = curve_adaptor.Value(u)
#         tangent_vector = curve_adaptor.DN(u, 1).Normalized()
#         tangent_dir = gp_Dir(tangent_vector)
        
#         perpendicular_plane = gp_Pln(pnt, tangent_dir)
#         perpendicular_face = BRepBuilderAPI_MakeFace(perpendicular_plane, -110, 110, -110, 110).Face()
#         display.DisplayShape(perpendicular_face, update=True, color='RED', transparency=0.5)
        
#         section = BRepAlgoAPI_Section(face, perpendicular_face, False)
#         section.ComputePCurveOn1(True)
#         section.Build()
        
#         if section.IsDone():
#             intersection_shape = section.Shape()
#             display.DisplayShape(intersection_shape, update=True, color='GREEN')
            
#             explorer = TopExp_Explorer(intersection_shape, TopAbs_EDGE)
#             edge = topods_Edge(explorer.Current())
#             curve = BRepAdaptor_Curve(edge)
            
#             num_points = 10
#             curve_length = GCPnts_AbscissaPoint.Length(curve, curve.FirstParameter(), curve.LastParameter())
#             points = GCPnts_UniformAbscissa(curve, curve_length / num_points, curve.FirstParameter())
            
#             for j in range(1, num_points + 1):
#                 param = points.Parameter(j)
#                 point = curve.Value(param)
#                 display.DisplayShape(point, update=True, color='YELLOW')
                
#                 surface_handle = BRep_Tool.Surface(face)
#                 projector = GeomAPI_ProjectPointOnSurf(point, surface_handle)
                
#                 if projector.NbPoints() > 0:
#                     u, v = projector.LowerDistanceParameters()
#                 else:
#                     print("No projection found for this point")
                    
#                 props.SetParameters(u, v)
#                 if props.IsCurvatureDefined():
#                     normal = props.Normal()
#                 if face.Orientation() == TopAbs_FACE:
#                     normal.Reverse()
                    
#                 with open('cylinder'  + '.txt', 'a') as outfile:
#                     outfile.write('\t' + str(face_index) + '\t' + str(path_index) +
#                                   '\t' + format(point.X(), ".2f") + '\t' + format(point.Y(), ".2f") + '\t' + format(point.Z(), ".2f") +
#                                   '\t' + format(normal.X(), ".2f") + '\t' + format(normal.Y(), ".2f") + '\t' + format(normal.Z(), ".2f") + '\n')
        
#                 normal_vector = gp_Vec(normal.XYZ())
#                 pnt_as_vec = gp_Vec(point.X(), point.Y(), point.Z())
#                 start = pnt_as_vec + normal_vector*10
#                 pnt_start = gp_Pnt(start.X(), start.Y(), start.Z())
        
#                 display.DisplayVector(normal_vector, point)
# start_display()

In [64]:
# ##給定一方向而生成線段

# from OCC.Display.SimpleGui import init_display
# from OCC.Core.gp import gp_Pnt, gp_Vec, gp_Dir
# from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge

# # 初始化显示
# display, start_display, add_menu, add_function_to_menu = init_display()

# # 起点
# start_point = gp_Pnt(0, 0, 0)

# # 方向和长度
# direction = gp_Dir(1, 0, 0)  # 沿 X 方向
# length = 5.0  # 线段长度

# # 计算结束点
# end_point = gp_Pnt(start_point.X() + length * direction.X(),
#                    start_point.Y() + length * direction.Y(),
#                    start_point.Z() + length * direction.Z())

# # 创建线段
# edge = BRepBuilderAPI_MakeEdge(start_point, end_point).Edge()

# # 显示线段
# display.DisplayShape(edge, update=True, color='RED')

# # 适应视图
# display.FitAll()

# # 开始显示
# start_display()